In [100]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Github/frankfurt_trees

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Github/frankfurt_trees


In [101]:
# Imports
import numpy as np
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format

from sklearn.linear_model import LinearRegression

import datetime
import re

# Load dataset
Public dataset provided by the City of Frankfurt as part of their Open Data initiative. Documentation can be found at: https://www.offenedaten.frankfurt.de/dataset/baumkataster-frankfurt-am-main

## Step 1: Load and clean 2016 and 2021 data

### A: 2016 data

In [102]:
trees_2016 = pd.read_csv('tree_register_frankfurt_2016.csv', sep=';', decimal=',')
trees_2016.rename(columns={'Gattung/Art/Deutscher Name': 'Gattung_2016', 'Objekt': 'Objekt_2016', 'Pflanzjahr': 'Pflanzjahr_2016', 'Kronendurchmesser': 'Kronendurchmesser_2016'}, inplace=True)
trees_2016['Alter_2016'] = 2016 - trees_2016['Pflanzjahr_2016']

In [103]:
trees_2016.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118403 entries, 0 to 118402
Data columns (total 8 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   Gattung_2016            118403 non-null  object 
 1   Baumnummer              118403 non-null  object 
 2   Objekt_2016             118403 non-null  object 
 3   Pflanzjahr_2016         118403 non-null  int64  
 4   Kronendurchmesser_2016  118403 non-null  float64
 5   HOCHWERT                118403 non-null  float64
 6   RECHTSWERT              118403 non-null  float64
 7   Alter_2016              118403 non-null  int64  
dtypes: float64(3), int64(2), object(3)
memory usage: 7.2+ MB


In [104]:
trees_2016.describe()

,Pflanzjahr_2016,Kronendurchmesser_2016,HOCHWERT,RECHTSWERT,Alter_2016
count,"118,403.00","118,403.00","118,403.00","118,403.00","118,403.00"
mean,"1,978.97",6.69,"5,552,953.08","475,244.51",37.03
std,24.03,3.87,"3,285.37","4,250.44",24.03
min,"1,645.00",2.00,"5,545,117.07","463,163.20",-1.00
25%,"1,970.00",4.00,"5,550,427.95","472,715.22",21.00
50%,"1,982.00",6.00,"5,552,600.68","475,218.65",34.00
75%,"1,995.00",9.00,"5,555,164.52","478,201.08",46.00
max,"2,017.00",63.00,"5,563,638.60","485,360.67",371.00


In [105]:
trees_2016.head(20)

,Gattung_2016,Baumnummer,Objekt_2016,Pflanzjahr_2016,Kronendurchmesser_2016,HOCHWERT,RECHTSWERT,Alter_2016
0,"Platanus x hispanica, Gewoehnliche Platane",1,Ackermannstrasse,1920,8.00,"5,549,510.90","473,366.24",96
1,"Platanus x hispanica, Gewoehnliche Platane",2,Ackermannstrasse,1920,8.00,"5,549,517.24","473,363.46",96
2,"Platanus x hispanica, Gewoehnliche Platane",3,Ackermannstrasse,1920,8.00,"5,549,524.39","473,360.15",96
3,"Platanus x hispanica, Gewoehnliche Platane",4,Ackermannstrasse,1920,8.00,"5,549,531.00","473,357.24",96
4,"Platanus x hispanica, Gewoehnliche Platane",5,Ackermannstrasse,1920,8.00,"5,549,537.75","473,354.48",96
5,"Platanus x hispanica, Gewoehnliche Platane",6,Ackermannstrasse,1920,8.00,"5,549,544.09","473,351.43",96
6,"Platanus x hispanica, Gewoehnliche Platane",7,Ackermannstrasse,1920,8.00,"5,549,550.83","473,348.26",96
7,"Platanus x hispanica, Gewoehnliche Platane",8,Ackermannstrasse,1920,8.00,"5,549,557.05","473,345.48",96
8,"Platanus x hispanica, Gewoehnliche Platane",9,Ackermannstrasse,1920,8.00,"5,549,563.93","473,342.70",96
9,"Platanus x hispanica, Gewoehnliche Platane",10,Ackermannstrasse,1920,8.00,"5,549,570.53","473,339.92",96


In [106]:
# The data is supposed to be from September 2017, so I guess this one was mislabeled or entered before it was planted?
trees_2016[trees_2016['Alter_2016'] < 0]

,Gattung_2016,Baumnummer,Objekt_2016,Pflanzjahr_2016,Kronendurchmesser_2016,HOCHWERT,RECHTSWERT,Alter_2016
112216,"Prunus cerasifera -Nigra-, Blut-Pflaume",15,Am Hennsee,2017,2.50,"5,552,326.61","483,260.02",-1


In [107]:
# Missing value strategy: Linear regression (complete overkill, but why not)
X = trees_2016[(trees_2016['Gattung_2016'] == 'Prunus cerasifera -Nigra-, Blut-Pflaume') & (trees_2016.index != 112216)][['Kronendurchmesser_2016']]
y = trees_2016[(trees_2016['Gattung_2016'] == 'Prunus cerasifera -Nigra-, Blut-Pflaume') & (trees_2016.index != 112216)]['Alter_2016']
reg = LinearRegression().fit(X, y)
print(f'R2 of regression of tree age on tree characteristics: {reg.score(X, y)}')
pred_age = reg.predict(trees_2016[trees_2016.index == 112216][['Kronendurchmesser_2016']])
print(f'Predicted age of tree #15: {pred_age}')
trees_2016.loc[[112216], ['Alter_2016']] = pred_age
print(trees_2016[trees_2016.index == 112216])

R2 of regression of tree age on tree characteristics: 0.44610339458097936
Predicted age of tree #15: [16.91244608]
                                   Gattung_2016 Baumnummer Objekt_2016  \
112216  Prunus cerasifera -Nigra-, Blut-Pflaume         15  Am Hennsee   

        Pflanzjahr_2016  Kronendurchmesser_2016     HOCHWERT  RECHTSWERT  \
112216             2017                    2.50 5,552,326.61  483,260.02   

        Alter_2016  
112216       16.91  


### B: 2021 data

In [108]:
trees_2021 = pd.read_csv('tree_register_frankfurt_2021.csv', sep=';', decimal=',')
trees_2021.rename(columns={'PFLANZJAHR': 'Pflanzjahr_2021', 'GA_LANG': 'Gattung_2021', 'KR_DURCHM': 'Kronendurchmesser_2021', 'BAUMNUMMER': 'Baumnummer', 'ST_UMFANG': 'Stammumfang_2021', 'BAUMHOEHE': 'Baumhoehe_2021', 'ST_DURCHM': 'Stammdurchmesser_2021','Pflanzjahr': 'Pflanzjahr_2021'}, inplace=True)
trees_2021['Alter_2021'] = 2021 - trees_2021['Pflanzjahr_2021']

In [109]:
trees_2021.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159187 entries, 0 to 159186
Data columns (total 17 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   Baumnummer              159187 non-null  object 
 1   HOCHWERT                159187 non-null  float64
 2   RECHTSWERT              159187 non-null  float64
 3   GATTUNGART              159182 non-null  object 
 4   GATTUNG                 159187 non-null  object 
 5   Gattung_2021            159187 non-null  object 
 6   Kronendurchmesser_2021  159187 non-null  float64
 7   Stammumfang_2021        159187 non-null  float64
 8   GEBIET                  159187 non-null  object 
 9   STRASSE                 159187 non-null  object 
 10  STANDORT                60122 non-null   object 
 11  Baumhoehe_2021          159187 non-null  int64  
 12  Stammdurchmesser_2021   159187 non-null  int64  
 13  Pflanzjahr_2021         159187 non-null  int64  
 14  BAUM_STATU          

In [110]:
trees_2021.describe()

,HOCHWERT,RECHTSWERT,Kronendurchmesser_2021,Stammumfang_2021,Baumhoehe_2021,Stammdurchmesser_2021,Pflanzjahr_2021,Kr_r,Alter_2021
count,"159,187.00","159,187.00","159,187.00","159,187.00","159,187.00","159,187.00","159,187.00","159,187.00","159,187.00"
mean,"5,552,877.00","475,348.65",7.10,106.45,12.74,33.91,"1,979.56",3.55,41.44
std,"3,390.37","4,185.87",4.04,73.51,6.15,23.40,25.55,2.02,25.55
min,"5,545,112.89","463,162.72",2.00,0.00,0.00,0.00,"1,700.00",1.00,-69.00
25%,"5,550,354.20","472,880.41",4.00,53.00,8.00,17.00,"1,968.00",2.00,24.00
50%,"5,552,681.20","475,550.45",6.00,89.00,12.00,28.00,"1,982.00",3.00,39.00
75%,"5,554,912.28","478,147.98",9.00,141.00,17.00,45.00,"1,997.00",4.50,53.00
max,"5,563,899.76","485,429.88",40.00,"4,457.00",67.00,"1,418.00","2,090.00",20.00,321.00


In [122]:
trees_2021.head(20)

,Baumnummer,HOCHWERT,RECHTSWERT,GATTUNGART,GATTUNG,Gattung_2021,Kronendurchmesser_2021,Stammumfang_2021,GEBIET,STRASSE,STANDORT,Baumhoehe_2021,Stammdurchmesser_2021,Pflanzjahr_2021,BAUM_STATU,Kr_r,Alter_2021,ID
0,1,"5,549,510.90","473,366.24",G0444,Platanus,"Platanus acerifolia, Gewöhnliche Platane",8.00,141.00,Straßen,Ackermannstrasse,Alte BNr. 1,7,45,1920,NaN,4.00,101.00,5549510.9473366.2391
1,2,"5,549,517.24","473,363.46",G0444,Platanus,"Platanus acerifolia, Gewöhnliche Platane",8.00,141.00,Straßen,Ackermannstrasse,Alte BNr. 3,7,45,1920,NaN,4.00,101.00,5549517.24473363.4592
2,3,"5,549,524.39","473,360.15",G0444,Platanus,"Platanus acerifolia, Gewöhnliche Platane",8.00,138.00,Straßen,Ackermannstrasse,Alte BNr. 5,7,44,1920,NaN,4.00,101.00,5549524.39473360.1493
3,5,"5,549,537.75","473,354.48",G0444,Platanus,"Platanus acerifolia, Gewöhnliche Platane",8.00,141.00,Straßen,Ackermannstrasse,Alte BNr. 9,7,45,1920,NaN,4.00,101.00,5549537.75473354.4795
4,6,"5,549,544.09","473,351.43",G0444,Platanus,"Platanus acerifolia, Gewöhnliche Platane",8.00,132.00,Straßen,Ackermannstrasse,Alte BNr. 11,7,42,1920,NaN,4.00,101.00,5549544.09473351.4296
5,7,"5,549,550.83","473,348.26",G0444,Platanus,"Platanus acerifolia, Gewöhnliche Platane",8.00,138.00,Straßen,Ackermannstrasse,Alte BNr. 13,7,44,1920,NaN,4.00,101.00,5549550.83473348.2597
6,8,"5,549,557.05","473,345.48",G0444,Platanus,"Platanus acerifolia, Gewöhnliche Platane",8.00,144.00,Straßen,Ackermannstrasse,Alte BNr. 15,7,46,1920,NaN,4.00,101.00,5549557.05473345.4798
7,9,"5,549,563.93","473,342.70",G0444,Platanus,"Platanus acerifolia, Gewöhnliche Platane",8.00,138.00,Straßen,Ackermannstrasse,Alte BNr. 17,7,44,1920,NaN,4.00,101.00,5549563.93473342.6999
8,10,"5,549,570.53","473,339.92",G0444,Platanus,"Platanus acerifolia, Gewöhnliche Platane",8.00,160.00,Straßen,Ackermannstrasse,Alte BNr. 19,7,51,1920,NaN,4.00,101.00,5549570.53473339.91910
9,11,"5,549,577.28","473,337.01",G0444,Platanus,"Platanus acerifolia, Gewöhnliche Platane",6.00,85.00,Straßen,Ackermannstrasse,Alte BNr. 21,7,27,1970,NaN,3.00,51.00,5549577.28473337.00911


In [112]:
# Not sure what the "Alte BNr." does - I tried substituting that for the current Baumnummer but this did not improve the match between the two datasets (quite the reverse, actually)
#trees_2021['Alte BNr.'] = trees_2021['STANDORT'].astype(str).apply(lambda x: re.search('Alte BNr. (\d+)*', x).group(1) if re.search('Alte BNr. (\d+)*', x) is not None else None)
#trees_2021['Baumnummer'] = np.where(~trees_2021['Alte BNr.'].isnull(), trees_2021['Alte BNr.'], trees_2021['Baumnummer'])

In [113]:
# There seems to be one wrong entry for the year the tree was planted in - this is being replaced with a predicted value based on a linear regression (complete overkill, but why not)
trees_2021[trees_2021['Alter_2021'] < 0]

,Baumnummer,HOCHWERT,RECHTSWERT,GATTUNGART,GATTUNG,Gattung_2021,Kronendurchmesser_2021,Stammumfang_2021,GEBIET,STRASSE,STANDORT,Baumhoehe_2021,Stammdurchmesser_2021,Pflanzjahr_2021,BAUM_STATU,Kr_r,Alter_2021
158555,24,"5,551,939.70","468,911.04",G0310,Juglans,"Juglans regia, Echte Walnuss",12.00,88.00,Grünanlagen,Gruenzug Am Sulzbach,NaN,12,28,2090,NaN,6.00,-69


In [114]:
X = trees_2021[(trees_2021['GATTUNGART'] == 'G0310') & (trees_2021.index != 158555)][['Kronendurchmesser_2021', 'Stammumfang_2021', 'Baumhoehe_2021', 'Stammdurchmesser_2021']]
y = trees_2021[(trees_2021['GATTUNGART'] == 'G0310') & (trees_2021.index != 158555)]['Alter_2021']
reg = LinearRegression().fit(X, y)
print(f'R2 of regression of tree age on tree characteristics: {reg.score(X, y)}')
pred_age = reg.predict(trees_2021[trees_2021.index == 158555][['Kronendurchmesser_2021', 'Stammumfang_2021', 'Baumhoehe_2021', 'Stammdurchmesser_2021']])
print(f'Predicted age of tree #24: {pred_age}')
trees_2021.loc[[158555], ['Alter_2021']] = pred_age
print(trees_2021[trees_2021.index == 158555])

R2 of regression of tree age on tree characteristics: 0.7713434476370453
Predicted age of tree #24: [36.72619309]
       Baumnummer     HOCHWERT  RECHTSWERT GATTUNGART  GATTUNG  \
158555         24 5,551,939.70  468,911.04      G0310  Juglans   

                        Gattung_2021  Kronendurchmesser_2021  \
158555  Juglans regia, Echte Walnuss                   12.00   

        Stammumfang_2021       GEBIET               STRASSE STANDORT  \
158555             88.00  Grünanlagen  Gruenzug Am Sulzbach      NaN   

        Baumhoehe_2021  Stammdurchmesser_2021  Pflanzjahr_2021 BAUM_STATU  \
158555              12                     28             2090        NaN   

        Kr_r  Alter_2021  
158555  6.00       36.73  


## Step 2: Combine datasets
Merge the 2016 and 2021 datasets

In [115]:
# Generate a unique ID by combining Baumnummer and coordinates
trees_2016['ID'] = trees_2016['HOCHWERT'].astype(str) + trees_2016['RECHTSWERT'].astype(str) + trees_2016['Baumnummer']
print(f'Assigned {len(trees_2016["ID"].unique())} unique IDs to 2016 dataset ({len(trees_2016.index)} total records in dataset)')
trees_2021['ID'] = trees_2021['HOCHWERT'].astype(str) + trees_2021['RECHTSWERT'].astype(str) + trees_2021['Baumnummer']
print(f'Assigned {len(trees_2021["ID"].unique())} unique IDs to 2021 dataset ({len(trees_2021.index)} total records in dataset)')

Assigned 118403 unique IDs to 2016 dataset (118403 total records in dataset)
Assigned 159187 unique IDs to 2021 dataset (159187 total records in dataset)


In [116]:
trees_combined = pd.merge(left=trees_2016[['ID', 'Gattung_2016', 'Objekt_2016', 'Pflanzjahr_2016', 'Kronendurchmesser_2016', 'Alter_2016']], right=trees_2021[['ID', 'Gattung_2021', 'Kronendurchmesser_2021', 'Stammumfang_2021', 'Baumhoehe_2021', 'Stammdurchmesser_2021', 'Pflanzjahr_2021', 'Alter_2021']], on='ID', how='outer')

In [117]:
trees_combined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 183153 entries, 0 to 183152
Data columns (total 13 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   ID                      183153 non-null  object 
 1   Gattung_2016            118403 non-null  object 
 2   Objekt_2016             118403 non-null  object 
 3   Pflanzjahr_2016         118403 non-null  float64
 4   Kronendurchmesser_2016  118403 non-null  float64
 5   Alter_2016              118403 non-null  float64
 6   Gattung_2021            159187 non-null  object 
 7   Kronendurchmesser_2021  159187 non-null  float64
 8   Stammumfang_2021        159187 non-null  float64
 9   Baumhoehe_2021          159187 non-null  float64
 10  Stammdurchmesser_2021   159187 non-null  float64
 11  Pflanzjahr_2021         159187 non-null  float64
 12  Alter_2021              159187 non-null  float64
dtypes: float64(9), object(4)
memory usage: 19.6+ MB


In [118]:
trees_combined.describe()

,Pflanzjahr_2016,Kronendurchmesser_2016,Alter_2016,Kronendurchmesser_2021,Stammumfang_2021,Baumhoehe_2021,Stammdurchmesser_2021,Pflanzjahr_2021,Alter_2021
count,"118,403.00","118,403.00","118,403.00","159,187.00","159,187.00","159,187.00","159,187.00","159,187.00","159,187.00"
mean,"1,978.97",6.69,37.03,7.10,106.45,12.74,33.91,"1,979.56",41.44
std,24.03,3.87,24.03,4.04,73.51,6.15,23.40,25.55,25.55
min,"1,645.00",2.00,0.00,2.00,0.00,0.00,0.00,"1,700.00",0.00
25%,"1,970.00",4.00,21.00,4.00,53.00,8.00,17.00,"1,968.00",24.00
50%,"1,982.00",6.00,34.00,6.00,89.00,12.00,28.00,"1,982.00",39.00
75%,"1,995.00",9.00,46.00,9.00,141.00,17.00,45.00,"1,997.00",53.00
max,"2,017.00",63.00,371.00,40.00,"4,457.00",67.00,"1,418.00","2,090.00",321.00


In [119]:
# Note row #3 - this seems to be a case where a new tree was planted, replacing an old one (died, presumably) and was assigned the same Baumnummer
trees_combined.head(20)

,ID,Gattung_2016,Objekt_2016,Pflanzjahr_2016,Kronendurchmesser_2016,Alter_2016,Gattung_2021,Kronendurchmesser_2021,Stammumfang_2021,Baumhoehe_2021,Stammdurchmesser_2021,Pflanzjahr_2021,Alter_2021
0,5549510.9473366.2391,"Platanus x hispanica, Gewoehnliche Platane",Ackermannstrasse,"1,920.00",8.00,96.00,"Platanus acerifolia, Gewöhnliche Platane",8.00,141.00,7.00,45.00,"1,920.00",101.00
1,5549517.24473363.4592,"Platanus x hispanica, Gewoehnliche Platane",Ackermannstrasse,"1,920.00",8.00,96.00,"Platanus acerifolia, Gewöhnliche Platane",8.00,141.00,7.00,45.00,"1,920.00",101.00
2,5549524.39473360.1493,"Platanus x hispanica, Gewoehnliche Platane",Ackermannstrasse,"1,920.00",8.00,96.00,"Platanus acerifolia, Gewöhnliche Platane",8.00,138.00,7.00,44.00,"1,920.00",101.00
3,5549531.0473357.2394,"Platanus x hispanica, Gewoehnliche Platane",Ackermannstrasse,"1,920.00",8.00,96.00,"Platanus acerifolia, Gewöhnliche Platane",2.00,25.00,5.00,8.00,"2,021.00",0.00
4,5549537.75473354.4795,"Platanus x hispanica, Gewoehnliche Platane",Ackermannstrasse,"1,920.00",8.00,96.00,"Platanus acerifolia, Gewöhnliche Platane",8.00,141.00,7.00,45.00,"1,920.00",101.00
5,5549544.09473351.4296,"Platanus x hispanica, Gewoehnliche Platane",Ackermannstrasse,"1,920.00",8.00,96.00,"Platanus acerifolia, Gewöhnliche Platane",8.00,132.00,7.00,42.00,"1,920.00",101.00
6,5549550.83473348.2597,"Platanus x hispanica, Gewoehnliche Platane",Ackermannstrasse,"1,920.00",8.00,96.00,"Platanus acerifolia, Gewöhnliche Platane",8.00,138.00,7.00,44.00,"1,920.00",101.00
7,5549557.05473345.4798,"Platanus x hispanica, Gewoehnliche Platane",Ackermannstrasse,"1,920.00",8.00,96.00,"Platanus acerifolia, Gewöhnliche Platane",8.00,144.00,7.00,46.00,"1,920.00",101.00
8,5549563.93473342.6999,"Platanus x hispanica, Gewoehnliche Platane",Ackermannstrasse,"1,920.00",8.00,96.00,"Platanus acerifolia, Gewöhnliche Platane",8.00,138.00,7.00,44.00,"1,920.00",101.00
9,5549570.53473339.91910,"Platanus x hispanica, Gewoehnliche Platane",Ackermannstrasse,"1,920.00",8.00,96.00,"Platanus acerifolia, Gewöhnliche Platane",8.00,160.00,7.00,51.00,"1,920.00",101.00
